# ML Pipeline Preparation

## 1. Import libraries and load data from database.

    Import Python libraries
    Load dataset from database with read_sql_table
    Define feature and target variables X and Y

In [1]:
# import libraries
import sqlite3
from sqlalchemy import create_engine
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pickle

import numpy as np
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer

import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /home/hugo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/hugo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/hugo/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /home/hugo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///messages_db.db')
df = pd.read_sql_table('messages', engine)

# cleaning the dataframe
df[df.columns[4:]] = df[df.columns[4:]].replace(2, np.NaN)
df.dropna(inplace=True)

X = df['message']
Y = df[df.columns[4:]]

In [3]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12411,14671,Ghambat: Khairpur: 100s of villages submerged ...,GHAMBAT: KHAIRPUR: 100S GAVON MEIN PAANI MEY J...,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
12412,14673,Ghambat: Sindh: Volunteers needed : @ http://b...,Ghambat: Sindh: Razakaroon ki zaroorat hai : @...,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12413,14675,"shikarpur: Volunteers needed, food and medical...",shikarpur mein swanyasevak ki avyashaktha he.k...,direct,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
12414,14676,"Dadu: Needs water, food, medicines. boats","JO KO KHANA, PAANI, DAWAYIAN, NAOV CHAHIYE.",direct,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


## 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stopwords.words("english")]
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

## 3. Build a machine learning pipeline

This machine pipeline should take in the message column as input and output classification results on the other 36 categories in the dataset. You may find the MultiOutputClassifier helpful for predicting multiple target variables

In [5]:
def build_model():
    pipeline = Pipeline([

        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
        
    ])
    
    parameters = {
    
        'clf__estimator__n_estimators': [100, 200],
    
    }

    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

## 4. Train pipeline

* Split data into train and test sets
* Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

model = build_model()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## 5. Test your model

Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's classification_report on each.

In [7]:
#  target_names = y_test.columns
scores_label_dict = {}

for i in range(36):
    scores = []
    try:
        class_rep_dict = classification_report(
            y_test.to_numpy()[:,i], y_pred[:,i], target_names=['0', '1'], output_dict=True)['macro avg']
        scores.append(class_rep_dict['precision'])
        scores.append(class_rep_dict['recall'])
        scores.append(class_rep_dict['f1-score'])
    except:
        scores.append(0)
        scores.append(0)
        scores.append(0)        
    
    scores_label_dict[df.columns[4:][i]] = scores
    
scores_label_df = pd.DataFrame.from_dict(scores_label_dict)

/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/si

/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/si

In [8]:
scores_label_df.set_index(pd.Index(['precision', 'recall', 'f1-score']))

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
precision,0.544026,0.546386,0.499604,0.578834,0.470833,0.581342,0.488497,0.494255,0.498217,0,...,0.495246,0.492469,0.571428,0.569632,0.648795,0.497821,0.486477,0.497821,0.490488,0.528883
recall,0.515877,0.520325,0.500000,0.547276,0.499159,0.504440,0.499392,0.500000,0.500000,0,...,0.500000,0.499799,0.511329,0.506124,0.510047,0.500000,0.498734,0.500000,0.499798,0.509365
f1-score,0.475101,0.487002,0.499802,0.526773,0.484582,0.500097,0.493884,0.497111,0.499107,0,...,0.497611,0.496107,0.490299,0.505716,0.510913,0.498908,0.481198,0.498908,0.495099,0.461585


## 7. Test your model

Show the accuracy, precision, and recall of the tuned model.

Since this project focuses on code quality, process, and pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
def eval_metrics(ArrayL, ArrayP, col_names):
    """Evalute metrics of the ML pipeline model
    
    inputs:
    ArrayL: array. Array containing the real labels.
    ArrayP: array. Array containing predicted labels.
    col_names: list of strings. List containing names for each of the ArrayP fields.
       
    Returns:
    data_metrics: Contains accuracy, precision, recall 
    and f1 score for a given set of ArrayL and ArrayP labels.
    """
    metrics = []
    
    # Evaluate metrics for each set of labels
    for i in range(len(col_names)):
        accuracy = accuracy_score(ArrayL[:, i], ArrayP[:, i])
        precision = precision_score(ArrayL[:, i], ArrayP[:, i])
        recall = recall_score(ArrayL[:, i], ArrayP[:, i])
        f1 = f1_score(ArrayL[:, i], ArrayP[:, i])
        
        metrics.append([accuracy, precision, recall, f1])
    
    # store metrics
    metrics = np.array(metrics)
    data_metrics = pd.DataFrame(data = metrics, index = col_names, columns = ['Accuracy', 'Precision', 'Recall', 'F1'])
      
    return data_metrics    

Here we call the previous function to evaluate the model

In [10]:
col_names = list(Y.columns.values)

print(eval_metrics(np.array(y_test), y_pred, col_names))

/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hugo/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1245: UndefinedM

                        Accuracy  Precision    Recall        F1
related                 0.648574   0.671013  0.922342  0.776855
request                 0.637480   0.429078  0.138286  0.209162
offer                   0.999208   0.000000  0.000000  0.000000
aid_related             0.618463   0.518433  0.229826  0.318471
medical_help            0.940174   0.000000  0.000000  0.000000
medical_products        0.961173   0.200000  0.010526  0.020000
search_and_rescue       0.975832   0.000000  0.000000  0.000000
security                0.988510   0.000000  0.000000  0.000000
military                0.996434   0.000000  0.000000  0.000000
child_alone             1.000000   0.000000  0.000000  0.000000
water                   0.911648   0.000000  0.000000  0.000000
food                    0.848653   0.153846  0.005362  0.010363
shelter                 0.892631   0.071429  0.003861  0.007326
clothing                0.993265   0.000000  0.000000  0.000000
money                   0.985737   0.000

### 9. Exporting the model as a pickle file

In [12]:
pickle.dump(model, open('model.plk', 'wb'))